## Installation and import blocks

In [ ]:
%pip install seaborn
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install pm4py

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pm4py

## Data retrieving

In [7]:
original_events_log = pm4py.read_xes("../Assets/Assets/Road_Traffic_Fine_Management_Process.xes")
original_events_log.head()

Exception: File does not exist